In [1]:
from huggingface_hub import list_models

def list_most_popular(task: str):
    for rank, model in enumerate(
        list_models(filter=task, sort="downloads", direction=-1)
):
        if rank == 5:
            break
        print(f"{model.id}, {model.downloads}\n")

list_most_popular("text-classification")

distilbert-base-uncased-finetuned-sst-2-english, 40672289

cardiffnlp/twitter-roberta-base-sentiment, 9292338

MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli, 7907049

cardiffnlp/twitter-roberta-base-irony, 7023579

SamLowe/roberta-base-go_emotions, 6706653



In [2]:
list_most_popular("summarization")

facebook/bart-large-cnn, 4637417

t5-small, 2492451

t5-base, 1887661

sshleifer/distilbart-cnn-12-6, 715809

t5-large, 332854



In [7]:
customer_email = """
I hope this email finds you amidst an aura of understanding, despite the tangled mess of emotions swirling within me as I write to you. I am writing to pour my heart out about the recent unfortunate experience I had with one of your coffee machines that arrived ominously broken, evoking a profound sense of disbelief and despair.

To set the scene, let me paint you a picture of the moment I anxiously unwrapped the box containing my highly anticipated coffee machine. The blatant excitement coursing through my veins could rival the vigorous flow of coffee through its finest espresso artistry. However, what I discovered within broke not only my spirit but also any semblance of confidence I had placed in your esteemed brand.

Imagine, if you can, the utter shock and disbelief that took hold of me as I laid eyes on a disheveled and mangled coffee machine. Its once elegant exterior was marred by the scars of travel, resembling a war-torn soldier who had fought valiantly on the fields of some espresso battlefield. This heartbreaking display of negligence shattered my dreams of indulging in daily coffee perfection, leaving me emotionally distraught and inconsolable
"""  # created by GPT-3.5

from langchain import HuggingFaceHub

summarizer = HuggingFaceHub(
    repo_id="facebook/bart-large-cnn",
    model_kwargs={"temperature":0, "max_length":180}
)
def summarize(llm, text) -> str:
    return llm(f"Summarize this: {text}!")

summarize(summarizer, customer_email)

'A customer\'s coffee machine arrived ominously broken, evoking a profound sense of disbelief and despair. "This heartbreaking display of negligence shattered my dreams of indulging in daily coffee perfection, leaving me emotionally distraught and inconsolable," the customer writes. "I hope this email finds you amidst an aura of understanding, despite the tangled mess of emotions swirling within me as I write to you," he adds.'

In [9]:
from transformers import pipeline

sentiment_model = pipeline(
    task="sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment"
)

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: fc6e199a-08c9-4f25-b97a-59094063cf43)')' thrown while requesting HEAD https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment/resolve/main/model.safetensors.index.json
'(ReadTimeoutError("HTTPSConnectionPool(host='cdn-lfs.huggingface.co', port=443): Read timed out. (read timeout=10.0)"), '(Request ID: 7c802dba-7047-4327-b78a-561232c347e6)')' thrown while requesting GET https://cdn-lfs.huggingface.co/cardiffnlp/twitter-roberta-base-sentiment/c37a3484c55954cd75b336a85f1e0c023ae874f3a73b05d2418dd04828e293b1?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27pytorch_model.bin%3B+filename%3D%22pytorch_model.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1699734205&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY5OTczNDIwNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9j

In [14]:
print(sentiment_model(customer_email))

[{'label': 'LABEL_0', 'score': 0.5822020173072815}]


In [11]:
print(sentiment_model("I am so angry and sad, I want to kill myself!"))

[{'label': 'LABEL_0', 'score': 0.9788626432418823}]


In [12]:
print(sentiment_model("I am elated, I am so happy, this is the best thing that ever happened to me!"))

[{'label': 'LABEL_2', 'score': 0.9926880598068237}]


In [13]:
print(sentiment_model("I don't care. I guess it's ok, or not, I couldn't care one way or the other"))


[{'label': 'LABEL_1', 'score': 0.5958545207977295}]


In [10]:
from langchain.llms import VertexAI
from langchain import PromptTemplate, LLMChain

template = """Given this text, decide what is the issue the customer is concerned about. Valid categories are these:
* product issues
* delivery problems
* missing or late orders
* wrong product
* cancellation request
* refund or exchange
* bad support experience
* no clear reason to be upset

Text: {email}
Category:
"""
prompt = PromptTemplate(template=template, input_variables=["email"])
llm = VertexAI()
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose=True)
print(llm_chain.run(customer_email))



> Entering new LLMChain chain...
Prompt after formatting:
Given this text, decide what is the issue the customer is concerned about. Valid categories are these:
* product issues
* delivery problems
* missing or late orders
* wrong product
* cancellation request
* refund or exchange
* bad support experience
* no clear reason to be upset

Text: 
I hope this email finds you amidst an aura of understanding, despite the tangled mess of emotions swirling within me as I write to you. I am writing to pour my heart out about the recent unfortunate experience I had with one of your coffee machines that arrived ominously broken, evoking a profound sense of disbelief and despair.

To set the scene, let me paint you a picture of the moment I anxiously unwrapped the box containing my highly anticipated coffee machine. The blatant excitement coursing through my veins could rival the vigorous flow of coffee through its finest espresso artistry. However, what I discovered within broke not only my spi